## Qué hay acá?

Tomando los datos de 0_raw, tomando el texto en español, aplico un modelo preentrenado de BERT para obtener vectores de 768 dimensiones para cada texto. Estos vectores los guardo en 1_intermediate-vectores.

In [3]:
import pandas as pd

import torch
from pytorch_transformers import *

In [2]:
df = pd.read_excel('../data/0_raw/0_raw.xlsx')

df = df["texto-original"]

# drop duplicates
df = df.drop_duplicates()

print(df.shape)
df.head()

(36,)


0    Pobres #MasistasIgnorantes piensan que por and...
3    @DraInterleucina @LuisFerCamachoV @Elgobernado...
4    El tongo siempre fue maleante. Mandó a desvali...
5    Así justifica lo tan obvio 🙄como su supuesto a...
7                           @grupoeldeber #revocatongo
Name: texto-original, dtype: object

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
# apply tokenizer to tweets in df
tokenized = df.apply((lambda x: torch.tensor([tokenizer.encode(x, add_special_tokens=True)])))

In [8]:
# apply model to tokenized
last_hidden_states = tokenized.apply(lambda x: model(x)[0])

In [19]:
# compute average
average = last_hidden_states.apply(lambda x: torch.mean(x, dim=1))

In [20]:
average.head()

0    [[tensor(0.0941, grad_fn=<UnbindBackward0>), t...
3    [[tensor(0.1626, grad_fn=<UnbindBackward0>), t...
4    [[tensor(0.0650, grad_fn=<UnbindBackward0>), t...
5    [[tensor(-0.0970, grad_fn=<UnbindBackward0>), ...
7    [[tensor(0.0577, grad_fn=<UnbindBackward0>), t...
Name: texto-original, dtype: object

In [21]:
# convert to numpy
average = average.apply(lambda x: x.detach().numpy())
# dimension (1, 768) to (768,)
average = average.apply(lambda x: x[0])
average.head()

0    [0.09410583, -0.3834717, 0.04874893, -0.002748...
3    [0.16256577, -0.46596974, 0.65866065, 0.163009...
4    [0.064989425, -0.61723, 0.033867605, 0.2302689...
5    [-0.0970382, -0.96525264, 0.1961879, 0.2774996...
7    [0.0577212, -0.9065021, 0.48618448, 0.37824604...
Name: texto-original, dtype: object

In [25]:
average.iloc[0].shape

(768,)

In [27]:
# export to json
average.to_json('../data/1_intermediate/1_intermediate-vectores.json', orient='records', lines=True)